## Dataset and DataLoader

In [ ]:
import random
list1 = [1, 2, 3, 4, 5, 6, 7]

batch_size = 2
epoch = 2
shuffle = True


for ep in range(epoch):
  if shuffle:
    random.shuffle(list1)
  for i in range(0, len(list1), batch_size):
    batch_data = list1[i:i+batch_size]
    print(batch_data)


[3, 1]
[7, 5]
[6, 4]
[2]
[2, 1]
[3, 4]
[7, 6]
[5]


In [ ]:
import random
import numpy as np


class MyDataset:
    def __init__(self, all_data, batch_size, shuffle):
        self.all_data = all_data
        self.bs = batch_size
        self.shuffle = shuffle
        self.cursor = 0

    def __iter__(self):
        return DataLoader(self)
        # pass
        # if self.shuffle:
        #     random.shuffle(self.all_data)
        #     self.cursor = 0
        # return self
    
    def __len__(self):
        return len(self.all_data)

    # def __next__(self):
    #     if self.cursor >= len(self.all_data):
    #         raise StopIteration
    # 
    #     batch_data = self.all_data[self.cursor: self.cursor+self.bs]
    #     self.cursor += self.bs
    #     return batch_data


class DataLoader():
    def __init__(self, dataset):
        self.dataset = dataset
        self.index = [i for i in range(len(self.dataset))]
        if self.shuffle:
            random.shuffle(self.index)
        self.cursor = 0
    
    def __next__(self):
        if self.cursor >= len(self.dataset.all_data):
            raise StopIteration
        
        # shuffle the index and then pick them instead of shuffling the data
        index = self.dataset.all_data[self.cursor: self.cursor + self.dataset.bs]
        batch_data = self.dataset.all_data[index]
        self.cursor += self.dataset.bs
        return batch_data


if __name__ == '__main__':

    all_data = np.array([1, 2, 3, 4, 5, 6, 7])
    batch_size = 2
    shuffle = True
    epoch = 2

    dataset = MyDataset(all_data, batch_size, shuffle)

    for ep in range(epoch):
        for data in dataset:
            print(data)




## Logistics Regression

In [ ]:
import numpy as np

def sigmoid(x):
    return 1/(1+np.exp(-x))

if __name__ == "__main__":
    count = 0
    dogs = np.array([[8.9,12],[9,11],[10,13],[9.9,11.2],[12.2,10.1],[9.8,13],[8.8,11.2]],dtype = np.float32)   # 0
    cats = np.array([[3,4],[5,6],[3.5,5.5],[4.5,5.1],[3.4,4.1],[4.1,5.2],[4.4,4.4]],dtype = np.float32)        # 1

    labels = np.array([0]*7 + [1]* 7,dtype = np.int32).reshape(-1,1)

    X = np.vstack((dogs,cats))

    k = np.random.normal(0,1,size=(2,1))
    b = 0
    epoch = 1000
    lr = 0.05

    for e in range(epoch):
        p = X @ k + b
        pre = sigmoid(p)

        loss = -np.sum(labels * np.log(pre) + (1-labels) * np.log(1-pre))

        G = pre - labels
        delta_k = X.T @ G
        delta_b = np.sum(G)

        k = k - lr * delta_k
        b = b - lr * delta_b
        # print(loss)

    while count < 2:
        count += 1
        f1 = float(input('Please input the length of hair:'))
        f2 = float(input("Please input the length of legs:"))

        test_x = np.array([f1,f2]).reshape(1,2)
        p = sigmoid(test_x @ k + b )
        if p >0.5:
            print("CAT")
        else:
            print("DOG")




Please input the length of hair:5
Please input the length of legs:8
CAT
Please input the length of hair:9
Please input the length of legs:3
CAT


## BP

In [ ]:
import numpy as np
import struct
import matplotlib.pyplot as plt

def load_labels(file):  
    with open(file, "rb") as f:
        data = f.read()
    return np.asanyarray(bytearray(data[8:]), dtype=np.int32)


def load_images(file):  
    with open(file, "rb") as f:
        data = f.read()
    magic_number, num_items, rows, cols = struct.unpack(">iiii", data[:16])
    return np.asanyarray(bytearray(data[16:]), dtype=np.uint8).reshape(num_items, -1)


def make_one_hot(labels,class_num=10):
    result = np.zeros((len(labels),class_num))
    for index,lab in enumerate(labels):
        result[index][lab] = 1
    return result

def sigmoid(x):
    return 1/(1+np.exp(-x))

def softmax(x):
    ex = np.exp(x)
    sum_ex = np.sum(ex,axis=1,keepdims=True)
    return  ex/sum_ex



if __name__ == "__main__":
    train_datas = load_images("/content/drive/MyDrive/N_F/data/train-images.idx3-ubyte") / 255
    train_label = make_one_hot(load_labels("/content/drive/MyDrive/N_F/data/train-labels.idx1-ubyte"))

    test_datas = load_images("/content/drive/MyDrive/N_F/data/t10k-images.idx3-ubyte") / 255
    test_label = load_labels("/content/drive/MyDrive/N_F/data/t10k-labels.idx1-ubyte")

    epoch = 20
    batch_size = 200
    lr = 0.01

    hidden_num = 256
    w1 = np.random.normal(0,1,size=(784,hidden_num))
    w2 = np.random.normal(0,1,size=(hidden_num,10))


    batch_times = int(np.ceil(len(train_datas) / batch_size))
    for e in range(epoch):
        for batch_index in range(batch_times):
    
            batch_x = train_datas[batch_index*batch_size:(batch_index+1)*batch_size]
            batch_label = train_label[batch_index*batch_size:(batch_index+1)*batch_size]

           
            h = batch_x @ w1
            sig_h = sigmoid(h)
            p = sig_h @ w2
            pre = softmax(p)

      
            loss = -np.sum(batch_label * np.log(pre))/batch_size


            G2 = (pre - batch_label)/batch_size
            delta_w2 = sig_h.T @ G2
            delta_sig_h = G2 @ w2.T
            delta_h = delta_sig_h * sig_h * (1-sig_h)
            delta_w1 = batch_x.T @ delta_h


            w1 = w1 - lr * delta_w1
            w2 = w2 - lr * delta_w2

        h = test_datas @ w1
        sig_h = sigmoid(h)
        p = sig_h @ w2
        pre = softmax(p)
        pre = np.argmax(pre,axis=1)
        acc = np.sum(pre == test_label)/10000
        print(acc)